In [733]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import Dropout
import csv

In [723]:
#load data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
weather = pd.read_csv('weather.csv')
spray = pd.read_csv('spray.csv')
#merge data (only using station 1 so as not to double the data)
trw = train.merge(weather[weather['Station']==1], how='left', on='Date')
tsw = test.merge(weather[weather['Station']==1], how='left', on='Date')
print(trw.shape, tsw.shape)

(10506, 33) (116293, 32)


In [715]:
#since classes are unbalanced, oversample WnvPresent = 1
#trw = trw.append(trw[trw.WnvPresent==1])   #didn't make any difference

In [398]:
tsw.columns

Index(['Id', 'Date', 'Address', 'Species', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'Station', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint', 'WetBulb',
       'Heat', 'Cool', 'Sunrise', 'Sunset', 'CodeSum', 'Depth', 'Water1',
       'SnowFall', 'PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed',
       'ResultDir', 'AvgSpeed'],
      dtype='object')

In [656]:
print(trw.shape, tsw.shape)

(10506, 33) (116293, 32)


In [256]:
#calculate baseline
trw[trw.WnvPresent == 1].shape

(551, 33)

In [ ]:
############################
#FEATURE ENGINEERING
############################

In [624]:
#unsued features (random forest crossval score didn't improve):
#
# Xtr['Lat1'] = trw.Latitude.round(1)
# Xts['Lat1'] = tsw.Latitude.round(1)

# Xtr['Long1'] = trw.Longitude
# Xts['Long1'] = tsw.Longitude

# Xtr['DewPoint'] = trw.DewPoint
# Xts['DewPoint'] = tsw.DewPoint

# Xtr['StnPressure'] = trw['StnPressure']
# ytr = ytr.drop(Xtr[Xtr['StnPressure']=='M'].index)
# Xtr = Xtr.drop(Xtr[Xtr['StnPressure']=='M'].index)
# Xtr['StnPressure'] = Xtr['StnPressure'].astype(float)
# Xts['StnPressure'] = tsw['StnPressure'].astype(float)
# #
# Xtr['AvgSpeed'] = trw['AvgSpeed'].astype(float)
# Xts['AvgSpeed'] = tsw['AvgSpeed'].astype(float)


In [ ]:
#need work
#Xtr['PrecipTotal']= trw.PrecipTotal.apply(lambda x: 0.1 if x == 'T' else x)
#Xts['PrecipTotal']= tsw.PrecipTotal.apply(lambda x: 0.1 if x == 'T' else x)

In [ ]:
#clean data, eliminate nulls
#create dummies 
#engineer data data (day of year?, sunrise?)
#engineer geographic variables
#engineer weather variables (lagging data)
#what to do about the 50 per trap limit?

In [725]:
#get dummy variables for only for mosquito species that have Wnv 
tsw_species = pd.get_dummies(tsw['Species'])[['CULEX PIPIENS/RESTUANS','CULEX PIPIENS','CULEX RESTUANS']]
trw_species = pd.get_dummies(trw['Species'])[['CULEX PIPIENS/RESTUANS','CULEX PIPIENS','CULEX RESTUANS']]

In [726]:
ytr = trw.WnvPresent
print(ytr.shape)
#Build X 
Xtr = pd.DataFrame()
Xts = pd.DataFrame()
#
Xtr['Latitude'] = trw.Latitude
Xts['Latitude'] = tsw.Latitude
Xtr['Longitude'] = trw.Longitude
Xts['Longitude'] = tsw.Longitude
#
print(Xtr.shape, ytr.shape)
Xtr['Tmax'] = trw.Tmax.astype(float)
Xts['Tmax'] = tsw.Tmax.astype(float)
#
Xtr['CULEX PIPIENS/RESTUANS'] = trw_species['CULEX PIPIENS/RESTUANS']
Xts['CULEX PIPIENS/RESTUANS'] = tsw_species['CULEX PIPIENS/RESTUANS']
Xtr['CULEX PIPIENS'] = trw_species['CULEX PIPIENS']
Xts['CULEX PIPIENS'] = tsw_species['CULEX PIPIENS']
Xtr['CULEX RESTUANS'] = trw_species['CULEX RESTUANS']
Xts['CULEX RESTUANS'] = tsw_species['CULEX RESTUANS']
#
Xtr['DayOfYear'] = pd.to_datetime(trw['Date'], format='%Y-%m-%d').dt.dayofyear
Xts['DayOfYear'] = pd.to_datetime(tsw['Date'], format='%Y-%m-%d').dt.dayofyear
#
Xtr['WetBulb'] = trw.WetBulb
ytr = ytr.drop(Xtr[Xtr['WetBulb']=='M'].index)
Xtr = Xtr.drop(Xtr[Xtr['WetBulb']=='M'].index)
Xtr['WetBulb'] = Xtr['WetBulb'].astype(float)
Xts['WetBulb'] = tsw.WetBulb.astype(float)                   #didn't have any 'M values
#

print(Xtr.shape, ytr.shape)
print(Xts.shape)

(10506,)
(10506, 2) (10506,)
(10413, 8) (10413,)
(116293, 8)


In [473]:
#check dtypes
#print(Xtr.dtypes)
#print(ytr.dtypes)

In [474]:
#check for nulls
#print(Xtr.isnull().sum())
#print(Xts.isnull().sum())

In [41]:
#Xts.isnull().sum()

In [ ]:
#####################################
#BUILD MODELS
######################################

In [ ]:
##############
#Sklearn models
##############

In [736]:
####### ADABoost model
model = AdaBoostClassifier(n_estimators=100) 
scores = cross_val_score(model, Xtr, ytr, cv=3)
print(scores)
print(np.mean(scores))

[0.86117512 0.94670124 0.68674352]
0.8315399566311958


In [582]:
#######RANDOM FOREST model
#not necessary
#X_train, X_test, y_train, y_test = train_test_split(Xtr, ytr, test_size=0.30, random_state=12)
#run random forest with kfold (may not be necessary, but will give an estimate of variance)
model = RandomForestClassifier(max_features = 6, max_depth = 20) 
scores = cross_val_score(model, Xtr, ytr, cv=3)
print(scores)
print(np.mean(scores))

[0.62327189 0.93546528 0.63544669]
0.7313946196865916


In [735]:
#######GRADIENT BOOSTING model
#not necessary
#X_train, X_test, y_train, y_test = train_test_split(Xtr, ytr, test_size=0.30, random_state=12)
#run random forest with kfold (may not be necessary, but will give an estimate of variance)
model = GradientBoostingClassifier(max_features = 6, max_depth = 100) 
scores = cross_val_score(model, Xtr, ytr, cv=3)
print(scores)
print(np.mean(scores))

[0.58006912 0.94237972 0.44553314]
0.6559939944316514


In [737]:
#fit against full training set
model.fit(Xtr,ytr)
model.score(Xtr,ytr)

0.9470853740516662

In [729]:
#metrics.roc_auc_score(ytr,y_preds)
y_preds = model.predict_proba(Xtr)[:,1]
metrics.roc_auc_score(ytr,y_preds)

0.8521624737162313

In [ ]:
#####################################################

In [52]:
################################
#KERAS MODEL
##############################

In [704]:
#Create keras Model
X_train, X_test, y_train, y_test = train_test_split(Xtr, ytr, test_size=0.30, random_state=11)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
ss = StandardScaler()
X_train = ss.fit_transform(X_train)  #the scaler is fit only to the training data
X_test = ss.transform(X_test)

model = Sequential()

input_units = X_train.shape[1] #number of features in training set
hidden_units = input_units   #hidden layer has the same number of nodes as input

#first input layer
model.add(Dense(hidden_units            
                ,input_dim=input_units  
                ,activation='relu'
                #uncomment this to add L2 regularization
                #,kernel_regularizer=regularizers.l2(0.0001) 
               ))


#hidden layer (try with and without)
node_reduction = 0
model.add(Dense(hidden_units - node_reduction          
                ,input_dim=input_units  
                ,activation='relu'
                #,kernel_regularizer=regularizers.l2(0.0001) 
               ))
#model.add(Dropout(0.8))

#final layer
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy'
              ,optimizer='adam'
               #added later (not part of original solution
              ,metrics=['binary_accuracy']
             )

(7289, 8) (7289,)
(3124, 8) (3124,)


In [706]:
#Run Keras model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), 
               epochs=20, batch_size=None, verbose=1)

Train on 7289 samples, validate on 3124 samples
Epoch 1/20
7289/7289 [==============================] - 1s 197us/step - loss: 0.7301 - binary_accuracy: 0.6238 - val_loss: 0.3361 - val_binary_accuracy: 0.9494
Epoch 2/20
7289/7289 [==============================] - 0s 55us/step - loss: 0.2650 - binary_accuracy: 0.9461 - val_loss: 0.2157 - val_binary_accuracy: 0.9494
Epoch 3/20
7289/7289 [==============================] - 0s 55us/step - loss: 0.2145 - binary_accuracy: 0.9461 - val_loss: 0.1972 - val_binary_accuracy: 0.9494
Epoch 4/20
7289/7289 [==============================] - 0s 55us/step - loss: 0.2049 - binary_accuracy: 0.9461 - val_loss: 0.1918 - val_binary_accuracy: 0.9494
Epoch 5/20
7289/7289 [==============================] - 0s 55us/step - loss: 0.2009 - binary_accuracy: 0.9461 - val_loss: 0.1888 - val_binary_accuracy: 0.9494
Epoch 6/20
7289/7289 [==============================] - 0s 54us/step - loss: 0.1987 - binary_accuracy: 0.9461 - val_loss: 0.1871 - val_binary_accuracy: 0.94

In [ ]:
#add some visualization here

In [ ]:
##############################################
##############################################

In [ ]:
#####################################
#SCORE MODEL
####################################

In [694]:
#run model against the kaggle test dataset
test_preds = model.predict_proba(Xts)[:,1]

In [707]:
#for keras only
#test_preds = test_preds[:,0]
test_preds = model.predict_proba(Xts)
test_preds = test_preds[:,0]

In [545]:
#metrics.roc_auc_score(yts,test_preds)

In [730]:
#generate output file 
test_preds = model.predict_proba(Xts)[:,1]
output_file = pd.DataFrame({'Id':tsw.Id, 'WnvPresent':test_preds})  
#output_file.head()
csv_name = 'test_csv.csv'
output_file.to_csv(csv_name, index=False)
print(output_file.shape)

(116293, 2)


In [448]:
#output_file.WnvPres.value_counts()

In [ ]:
##########################################